In [ ]:
#import required libraries
from azure.ml import dsl

In [56]:
%load_ext autoreload
%autoreload 2

from components.prep import data_prep_component
from components.split_data import split_data_component
from components.train import train_component
from components.score import score_component
from components.eval import eval_component

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# define a pipeline with dsl component
@dsl.pipeline(
    name='A-training-pipeline',
    description='train model and evaluate model perf',
)
def training_pipeline_func(
    input_data,
    feature_list: str,
    label_col: str,
    learning_rate: float,
    n_estimators: int,
    test_size: float = 0.3,
    random_state: int = 42
    ):
    
    prep = data_prep_component(
        data_source=input_data
    )

    split = split_data_component(
        data_cooked=prep.outputs.data_cooked,
        test_size=test_size,
        random_state=random_state
    )

    train = train_component(
        train_data=split.outputs.data_train,
        feature_list=feature_list,
        label_col=label_col,
        learning_rate=learning_rate,
        n_estimators=n_estimators
    )

    score = score_component(
        test_data=split.outputs.data_test,
        model_input=train.outputs.model_output,
        feature_list=feature_list
    )
    
    eval = eval_component(
        scored_data=score.outputs.scored_data,
        label_col=label_col
    )
    return {
        'data_output': eval.outputs.eval_result,
    }
